In [1]:
!ngrok authtoken 1lXQip5p1qG5x06gvFFVZexKhuD_zxHcRWnGwde53rKQkQVw

/bin/bash: ngrok: command not found


In [2]:
!pip install streamlit
!pip install pyngrok

     |████████████████████████████████| 7.4MB 8.7MB/s 
     |████████████████████████████████| 163kB 43.8MB/s 
     |████████████████████████████████| 102kB 9.6MB/s 
     |████████████████████████████████| 112kB 39.3MB/s 
     |████████████████████████████████| 4.5MB 44.4MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 122kB 48.6MB/s 
  Created wheel for watchdog: filename=watchdog-1.0.1-cp36-none-any.whl size=72206 sha256=c2ab4937030a70d83589bed4e845596b844b329c464beaef46b7cb901a1255bd
  Stored in directory: /root/.cache/pip/wheels/d8/ce/d8/31a48288b5728794feda5ac479fa324cc1cde4398c29eff064
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=b0f89423f0432a5551957011cc2e7b4421177c1ade65496542d5449489c0cd59
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built watchdog blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.1

  Created wheel for pyngrok: filename=pyngrok-5.0.0-cp36-none-any.whl size=18781 sha256=fa183d9c8e5a716236df42e020a7dc7d4a901b7ce2531e6ae835029f9eec597f
  Stored in directory: /root/.cache/pip/wheels/95/df/23/af8dde08c3fcdc7b966adcacef48ab29aa3b0b1860df5d2b79
Successfully built pyngrok


In [3]:
!git clone https://github.com/1stDayHack/FDK.git
!pip install -r FDK/requirements.txt
!pip install matplotlib==3.1.3

Cloning into 'FDK'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 1939 (delta 21), reused 22 (delta 4), pack-reused 1892
Receiving objects: 100% (1939/1939), 487.14 MiB | 38.22 MiB/s, done.
Resolving deltas: 100% (217/217), done.
Checking out files: 100% (1600/1600), done.
     |████████████████████████████████| 7.6MB 6.2MB/s 
     |████████████████████████████████| 11.6MB 21.0MB/s 
     |████████████████████████████████| 49.3MB 89kB/s 
     |████████████████████████████████| 12.4MB 39.2MB/s 
     |████████████████████████████████| 778kB 44.8MB/s 
     |████████████████████████████████| 2.1MB 29.2MB/s 
     |████████████████████████████████| 276kB 43.5MB/s 
     |████████████████████████████████| 890kB 40.0MB/s 
     |████████████████████████████████| 1.1MB 37.8MB/s 
     |████████████████████████████████| 3.0MB 37.4MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
  Create

     |████████████████████████████████| 13.1MB 209kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.3.1
    Uninstalling matplotlib-3.3.1:
      Successfully uninstalled matplotlib-3.3.1


In [9]:
%%writefile app.py
from PIL import Image
import streamlit as st
import os, shutil, cv2
import matplotlib.pyplot as plt
import numpy as np

from matplotlib import cm
from PIL import Image

from FDK.src.core.depth import DepthEst
from FDK.src.core.utils import utils


st.set_page_config(
    page_title="St M.",
    page_icon=":wave:",
    layout="centered"
)

page_bg_img = '''
<style>
body {
background-color: lightyellow;
}
</style>
'''

st.markdown(page_bg_img,unsafe_allow_html=True)

depth = DepthEst(name="DeptEstimator",device="cpu")

def read_image(img_path):
    images = []

    dir_images = os.listdir(img_path)
    #dir_images.sort()

    for i in dir_images:
        img_ = Image.open(img_path + '/' + i)
        img_ = utils.pil_to_cv2(img_)
        images.append(img_)
    
    return images

    

st.title("Eye Care")
st.subheader("Provide images to check")

img_path = st.text_input("","None")

images = []

if img_path != "None":
    images = read_image(img_path)
    st.success("Uploaded.")

st.markdown('<hr>', unsafe_allow_html=True)
st.subheader("Main Display")

use_col_width = True

# Picture with 2 column
col1, col2 = st.beta_columns(2)

with col1:
    main_display_col1 = st.empty()

with col2:
    main_display_col2 = st.empty()

if img_path != "None":
    raw_display_image = utils.cv2_to_pil(images[0])
    main_display_col1 = main_display_col1.image(raw_display_image,use_column_width=use_col_width,caption="Raw",format="RGB")

    depth_output = depth.predict(images[0])
    depth_output = depth_output/depth_output.max()
    depth_output = Image.fromarray(np.uint8(cm.gist_earth(depth_output)*255)).convert("RGB")
    main_display_col2 = main_display_col2.image(depth_output,use_column_width=use_col_width,caption="Depth Map")


max_len = (len(images)-1) if images else 1
idx_to_run = st.slider("Preview Frame Selection",0,max_len)

if idx_to_run:
    raw_display_image = utils.cv2_to_pil(images[idx_to_run])
    main_display_col1.image(raw_display_image,use_column_width=use_col_width,caption="Raw",format="RGB")
    depth_output = depth.predict(images[idx_to_run])
    depth_output = depth_output/depth_output.max()
    depth_output = Image.fromarray(np.uint8(cm.gist_earth(depth_output)*255)).convert("RGB")
    main_display_col2.image(depth_output,use_column_width=use_col_width,caption="Depth Map")



st.markdown('<hr>',unsafe_allow_html=True)
st.subheader("Finished? Save Results and Clear Workspace!")


if st.button("Run All and Save"):

    st.info("This will create new folders in the current working dir to save.")

    if not os.path.isdir("depth_output"):
        os.mkdir("depth_output")

    for idx, i in enumerate(images):
        depth_output = depth.predict(i)
        depth_output = depth_output/depth_output.max()
        depth_output = Image.fromarray(np.uint8(cm.gist_earth(depth_output)*255)).convert("RGB")
        depth_output.save("depth_output/depth_{:05d}.jpg".format(idx))



Overwriting app.py


In [5]:
!ls

app.py	FDK  sample_data


In [6]:
!streamlit run app.py &>/dev/null&

In [11]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

<NgrokTunnel: "http://0aa21c41e15b.ngrok.io" -> "http://localhost:80">

In [12]:
!streamlit run --server.port 80 app.py >/dev/null

Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


In [ ]:
!pgrep streamlit

234
373
428
810
912
1317


In [ ]:
ngrok.kill()